In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns

import search

# Example 1: Target depends on two independent features being equal

Let 
$X_i$, $i=1, \dots, d$
be independent discrete random variables with
$X_i$ 
uniformly distributed on  
$\lbrace 0, \dots, m_i\rbrace$.


Let
$1\leq  k_0, k_1 \leq d$
be distint integers and let
$Y := 1\lbrace(X_{k_0} = X_{k_1}\rbrace$
be equal to $1$ when $X_{k_0}$ and $X_{k_1}$ are equal,
and zero otherwise.

Given samples 
$$
\left\lbrace
\left(
x^{(n)}_{1}, \dots , x^{(n)}_{d}, y^{(n)}
\right):
\,\,
n = 1,  2 \dots
\right\rbrace
$$
of the discrete random vector
$
(X_1, \dots, X_d, Y)
$
we want a
procedure that estimates which of the features
$i = 1, \dots, d$
is relevant for the prediction of $Y$. 
By construction,
the correct answer is the pair $\lbrace k_0, k_1\rbrace$,
and we can evaluate the accuracy of a procedure by comparing the returned feature selection against the expected pair $\lbrace k_0, k_1\rbrace$.

In [ ]:
k = 5
n = 2000
d = 30

In [ ]:
x0 = np.random.randint(k, size=(n, 1))
x1 = np.random.randint(k, size=(n, 1))
ms = np.random.randint(low=2, high=20, size = d-2)
others = [np.random.choice(m, size=(n, 1)) for m in ms]
all_ = np.concatenate(
    [x0, x1] + others,
    axis=1
)
y = np.asarray(x0[:, 0] == x1[:, 0], dtype=int) # k + x0 - x1 # np.asarray(x0 == x1, dtype=int)
permuter =  np.random.permutation(np.eye(d, dtype=int).T).T
x = np.array(all_ @ permuter, dtype=int)
expected_features = [np.argmax(permuter[0, :]), np.argmax(permuter[1, :])]

In [ ]:
assert np.all(x[:, expected_features[0]] == x0[:, 0])
assert np.all(x[:, expected_features[1]] == x1[:, 0])

In [ ]:
df = pd.DataFrame(x[:, expected_features], columns=[f'f{i}' for i in expected_features])
df['y'] = y
sns.scatterplot(df, x=f'f{expected_features[0]}', y=f'f{expected_features[1]}', hue='y')

In [ ]:
print(f'expected_features:\n{sorted(expected_features)}')#

## Comparison of three different feature selection methods

We compare three different methods for feature selection:

A. Selection using permutation sampling

B. Selection using single-marginal mutual information

C. Boruta

### A. Selection using permutation sampling

For random variables
$X$, $Y$,
let 
$I(X, Y) \geq 0$
be a non-negative measure of probabilitstic dependence,
such that 
$I(X, Y) = 0$ 
if and only if 
$X$ and $Y$ are independent.
Moreover, 
we assume that,
given three random varibales 
$X$, $Y$, $Z$,
we can informally interpret the inequality
$I(X,Y) \leq I(X,Z)$
as stating that 
$X$ is "more dependent" on $Z$ than it is on $Y$.

Let 
$X_i$, $i=1, \dots, d$
be the $d$ features to select from, 
and let $Y$ be the target.

Given a sample size $N$,
let 
$$
x_i = \lbrace x^{n}_i: n=1, \dots, N\rbrace
$$
be samples from $X_i$,
and 
$$
y = \lbrace y^{n}: n=1, \dots, N\rbrace
$$
be samples from $Y$.

For indices $i_1, i_2, \dots, i_k$, we let 
$I(x_{i_1}, \dots, x_{i_k}, y)$
be the measure of probabilistic dependence 
between
the empirical vector 
$(x_{i_1}, \dots, x_{i_p})$
sampled from
$(X_{i_1}, \dots, X_{i_p})$
and 
the empirical target
$y$
sampled from $Y$.

Let 
$\sigma^{(1)}, \dots , \sigma^{(p)}$
be $p$ permutations of length $d$. 
For
$\ell = 1, \dots, p$
and
$k = 1, \dots, d$
let
$$
I^{(\ell)}_{k} : =
I(x_{\sigma^{(\ell)}_{1}}, \dots, x_{\sigma^{(\ell)}_{k}}, y)
$$
We consider the maximisation problem
$$
\max
\left\lbrace
I^{(\ell)}_{k} :
\,\,
\ell = 1, \dots, p,
\,
k = 1, \dots, d
\right\rbrace.
$$
Let $\hat{\ell}$, $\hat{k}$ be the maximisers. 
Then,
the features selected by this procedures are
$$
\sigma^{(\hat{\ell})}_{1}, \dots, \sigma^{(\hat{\ell})}_{\hat{k}}.
$$

The feature selection procedure is then fully specified when we give 

1) A way to choose the $p$ permutations
$\sigma^{(1)}, \dots , \sigma^{(p)}$
in
$\mathfrak{S}_d$;

2) The measure of probabilistic dependence $I$.

#### 1) Choose $p$ permutations $\sigma^{(1)}, \dots, \sigma^{(p)}$ in $\mathfrak{S}_d$.

We use an approach ispired by 
[Plis et al. (2010)](https://ieeexplore.ieee.org/document/5693994)
and
[Mitchel et al. (2022)](https://arxiv.org/abs/2104.12199).


Let 
$$
f: S^{d-2} \longrightarrow \mathfrak{S}_d
$$
be a surjective continuous map 
from the unit sphere in $R^{d-1}$
to the symmetric group $
\mathfrak{S}_d$ of permutations of length $d$.

Assume that $f$ is such that
for every $\sigma$ in $\mathfrak{S}_d$,
the preimage
$f^{-1}(\sigma)$ 
is connected and 
there exists a non-empy,  open set 
$U_{\sigma} \subset S^{d-2}$
such that 
$U_{\sigma} \subset f^{-1}(\sigma)$.

The map $f$
alllows us to transfer 
the problem of choosing well distributed permutations
to 
the problems of choosing well distributed points on the sphere.
The latter is more well-studied 
and it simplifies our quest. 

Following
Mitchel et al. (2022),
we let 
$$
f(x) = \text{argsort} Px
$$
where $P$ in $M_{d\times (d-1)}(R)$ is the projection matrix
that maps points in
$R^{d-1}$
to points in $R^{d}$
lying on the hyperplane with normal vector
$\bar{n} = 1/\sqrt{d} \cdot \underbrace{(1, \dots, 1)}_{d}$

We sample points 
$x_1, x_2, \dots$
on $S^{d-2}$
by drawing 
$U_1, U_2, \dots$
from the Haar distribution on the special orthogonal group $SO(d-1)$:
every column $u_i^1, \dots,  u_i^{d-1}$ of any of these orthogonal matrices $U_i$ gives two 
antipodal points on the sphere
$x_i^{2k} = u_i^{k}$
and
$x_i^{2k-1} = - u_i^{k}$
for 
$k = 1, \dots, d-1$.

#### 2) Choose $I$

We choose
$I$
as the 
[adjusted mutual information](https://en.wikipedia.org/wiki/Adjusted_mutual_information),
an implementation of which is readily available from 
[sklearn.metrics.adjusted_mutual_info_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_mutual_info_score.html).

In [ ]:
%%time 
selected_features = search.select_features(
    x, y, 
    num_haar_samples=3,
)

In [ ]:
print(f'expected_features:\n{sorted(expected_features)}')
print(f'selected_features:\n{sorted(selected_features)}')

### B. Selection using single-marginal mutual information

For random variables
$X$, $Y$,
let 
$I(X, Y) \geq 0$
be a non-negative measure of probabilitstic dependence,
such that 
$I(X, Y) = 0$ 
if and only if 
$X$ and $Y$ are independent.
Moreover, 
we assume that,
given three random varibales 
$X$, $Y$, $Z$,
we can informally interpret the inequality
$I(X,Y) \leq I(X,Z)$
as stating that 
$X$ is "more dependent" on $Z$ than it is on $Y$.

Let 
$X_i$, $i=1, \dots, d$
be the $d$ features to select from, 
and let $Y$ be the target.

Given a sample size $N$,
let 
$$
x_i = \lbrace x^{n}_i: n=1, \dots, N\rbrace
$$
be samples from $X_i$,
and 
$$
y = \lbrace y^{n}: n=1, \dots, N\rbrace
$$
be samples from $Y$.

For indices $i_1, i_2, \dots, i_k$, we let 
$I(x_{i_1}, \dots, x_{i_k}, y)$
be the measure of probabilistic dependence 
between
the empirical vector 
$(x_{i_1}, \dots, x_{i_p})$
sampled from
$(X_{i_1}, \dots, X_{i_p})$
and 
the empirical target
$y$
sampled from $Y$.

We compute 
$I(x_i, y)$
for
$i = 1, \dots, d$.
We construct the vector
$$
a_1, \dots, a_d
$$
where
$$
a_i = \frac{I(x_i, y)} {\hat{I}}
$$
where
$\hat{I} = \max_{i}I(x_i, y)$.

Given a threshold of negligibility $\epsilon\geq 0$,
this procedure seleccts feature $i$ if and only if $a_i > \epsilon$.

We choose $I$ as the mutual information as implemented in
[sklearn.feature_selection.mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html).

In [ ]:
threhsold = .04

In [ ]:
%%time
mis = mutual_info_classif(x, y, discrete_features=True)
mis /= np.amax(mis)
mi_selection, = np.where(mis> threhsold)

In [ ]:
print(f'expected_features:\n{sorted(expected_features)}')
print(f'Single marginal Mi selection:\n{sorted(mi_selection)}')

In [ ]:
sns.barplot(x=mi_selection, y=mis[mi_selection])

### C. Selection with Boruta

In [ ]:
from arfs.feature_selection import allrelevant
from arfs.feature_selection.allrelevant import Leshy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
n_estimators = 'auto'
perc = 95
alpha = 0.05
importance = "shap"
two_step = True
max_iter = 100
random_state = None
verbose = 0
keep_weak = False

In [ ]:
xdf = pd.DataFrame(x, columns = [f'f{i}' for i in range(d)])
yser = pd.Series(y, name='y')

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, max_depth=8)

In [ ]:
leshy = Leshy(
    rf,
    n_estimators=n_estimators,
    perc=perc,
    alpha=alpha,
    importance=importance,
    two_step=two_step,
    max_iter=max_iter,
    random_state=random_state,
    verbose=verbose,
    keep_weak=keep_weak,
)

In [ ]:
%%time
leshy.fit(xdf, yser)

In [ ]:
leshy_selection = [int(col.replace('f', '')) for col in leshy.selected_features_]

In [ ]:
print(f'Expected features: {sorted(expected_features)}')
print(f'Boruta selection: {sorted(leshy_selection)}')